In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

import os
import torch
import matplotlib.pyplot as plt
from skimage.io import imread
import numpy as np
from skimage import img_as_ubyte
import imageio
import json
import cv2
import time
from PIL import Image
import math
import sys
import os
import glob
sys.path.append(os.path.abspath(''))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
def now_str():
    now = datetime.now()
    month = str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    sec = str(now.second)
    
    output = '[{:>02}/{:>02} {:>02}:{:>02}:{:>02}]'.format(month, day, hour, minute, sec)
    return output
def __output_log(path, strs):
    if not os.path.exists(path):
        with open(path, 'w+') as f:
            f.write(strs)
            f.close()
    else:
        with open(path, 'a+') as f:
            f.write(strs)
            f.close()
def get_path(func):  
    if type(func).__name__ == 'function' : 
        return func.__code__.co_filename
    else: 
        raise ValueError("'func' must be a function") 
print(now_str())


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[06/02 22:35:34]


# Load jsons: parameters and losses

In [44]:
in_dir = './10f_data/output/sphere/3_rgba_rgb_singleRes'
parameter_losses = [] # list of tuples
for setting_idx in range(90):
    input_dir = in_dir + '/setting_{}'.format(setting_idx)
    
    param_path = input_dir + '/parameters.json'
    with open(param_path, 'r') as f:
        params = json.load(f)
        f.close()

    loss_path = input_dir + '/losses.json'
    with open(loss_path, 'r') as f:
        loss = json.load(f)
        f.close()
    
    parameter_losses.append(tuple((params, loss)))
print(len(parameter_losses), 'settings & losses')

90 settings & losses


# Organize losses by each of parameters

In [50]:
param2loss = {} # dict of list
min_loss = 999
min_param_loss = -1
for pl in parameter_losses:
    p = pl[0]
    l = pl[1]['all'][-1]

    if l < min_loss:
        min_loss = l
        min_param_loss = pl
        
    for param_name, v in p.items():
        if param_name == 'index':
            continue
        if param_name in param2loss:
            if v in param2loss[param_name]:
                param2loss[param_name][v].append(l)
            else:
                param2loss[param_name][v] = [l]
        else:
            param2loss[param_name] = {v: [l]}
        
# sort
for k, v in param2loss.items():
    if len(v.keys()) > 1:
        print(k)
        for kk, vv in v.items():
            param2loss[k][kk] = sorted(vv)
            print('  {}: {} losses | min={:.4f}, max={:.4f}'.format(kk, len(vv), min(vv), max(vv)))
print()
print('min loss: ', min_loss)
print(min_param_loss[0])

resize_method
  pytorch3d: 45 losses | min=0.2365, max=0.8672
  interpolate: 45 losses | min=0.2123, max=1.1376
faces_per_pixel
  1: 18 losses | min=0.2123, max=0.8672
  2: 18 losses | min=0.3317, max=0.8532
  3: 18 losses | min=0.3453, max=1.1376
  4: 18 losses | min=0.3773, max=0.7760
  5: 18 losses | min=0.3731, max=0.6614
blur_radius
  0.0001: 30 losses | min=0.2123, max=0.6596
  0.001: 30 losses | min=0.3436, max=0.6675
  0.01: 30 losses | min=0.3317, max=1.1376
sigma
  0.0001: 30 losses | min=0.2123, max=1.1376
  0.001: 30 losses | min=0.2937, max=0.8672
  0.01: 30 losses | min=0.3317, max=0.6675

min loss:  0.21234816312789917
{'index': 45, 'nnParameter_type': 'verts_deform', 'image_size': 256, 'num_cams': 16, 'resize_method': 'interpolate', 'batch_size': 8, 'use_texture': True, 'texturemap_size': 16, 'loss_type': 'rgba', 'faces_per_pixel': 1, 'target_offset': 0.0, 'blur_radius': 0.0001, 'sigma': 0.0001, 'gamma': 0.0001, 'lr': 0.005, 'use_scheduler': True}
